# Aerogel bonding Final Project AI & ML 2024
## Importing Libraries & Reading Data

In [ ]:
#Importing Libraries 

import numpy as np 
import pandas as pd
import sklearn

#Reading Data Set
path = './aerogel_bonding.csv'
df = pd.read_csv(path)

## EDA & Data Pre-Processing
Here we will visualize the data, take into account important values and clean the dataset from irrelevant data

### Vizualising and Understaning Data Structure & Values

In [3]:
df.head()

,HolidaysTaken,PercentageOfCompletedTasks,CurrentJobDuration,RecentHolidaysTaken,RequestedProcessAmount,JobStatus,BondingRiskRating,TotalMaterialProcessed,ByproductRation,working_skills,...,TotalMaterialToProcess,WorkHistoryDuration,ApplicantAge,PriorExecutionDefaults,DifferentTasksCompleted,TotalChurnRisk,OtherCompaniesMaterialProcessed,BondingPeriod,trustability,MonthlyExecutions
0,4.0,NaN,6.0,1.0,51172.0,Employed,46.0,300388.0,0.273137,1.244120,...,14193.0,3.0,67.0,0.0,NaN,NaN,NaN,24.0,3.144274,440.0
1,1.0,0.323046,4.0,0.0,11246.0,Employed,54.0,299914.0,0.450387,2.228183,...,85355.0,28.0,52.0,NaN,0.0,0.232582,214559.0,36.0,3.704809,NaN
2,5.0,0.491574,3.0,1.0,14075.0,Employed,42.4,74687.0,0.325027,2.699264,...,14006.0,NaN,45.0,0.0,0.0,0.240812,60681.0,60.0,2.427195,171.0
3,4.0,0.108916,3.0,1.0,18957.0,Employed,40.8,47866.0,NaN,0.445854,...,13240.0,16.0,42.0,0.0,4.0,0.231520,34626.0,84.0,1.156431,212.0
4,NaN,0.174628,1.0,2.0,17902.0,Employed,51.0,18181.0,0.388317,1.940075,...,44217.0,28.0,50.0,0.0,0.0,0.214425,4812.0,48.0,3.185402,323.0


In [7]:
df.dtypes

HolidaysTaken                      float64
PercentageOfCompletedTasks         float64
CurrentJobDuration                 float64
RecentHolidaysTaken                float64
RequestedProcessAmount             float64
JobStatus                           object
BondingRiskRating                  float64
TotalMaterialProcessed             float64
ByproductRation                    float64
working_skills                     float64
CivilStatus                         object
dependability                      float64
MistakesLastYear                   float64
HighestEducationAttained            object
BondingSuccessful                  float64
ChurnRisk                          float64
ProcessedKilograms                 float64
SkillRating                        float64
ProcessingTimestamp                 object
WorkExperience                     float64
HistoricalBehavior                 float64
TotalMaterialToProcess             float64
WorkHistoryDuration                float64
ApplicantAg

### Checking Data Integrity

In [5]:
# Calculate missing values
missing_values = df.isnull().sum()

# Calculate total values (missing + non-missing)
total_values = df.count() + missing_values

# Calculate percentage of missing values and format with percentage sign
percentage_missing = (missing_values / total_values * 100).apply(lambda x: f'{x:.2f}%')

# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Total Values': total_values,
    'Percentage Missing': percentage_missing
})

# Print the summary
print("Summary:")
print(summary_df)

Summary:
                                 Missing Values  Total Values  \
HolidaysTaken                              1905         20000   
PercentageOfCompletedTasks                 2045         20000   
CurrentJobDuration                         2048         20000   
RecentHolidaysTaken                        1969         20000   
RequestedProcessAmount                     2075         20000   
JobStatus                                  2018         20000   
BondingRiskRating                          2050         20000   
TotalMaterialProcessed                     2043         20000   
ByproductRation                            1972         20000   
working_skills                             1966         20000   
CivilStatus                                1954         20000   
dependability                              1949         20000   
MistakesLastYear                           1987         20000   
HighestEducationAttained                   2063         20000   
BondingSuccessfu